In [1]:
!git clone https://github.com/jayded/evidence-inference.git

fatal: destination path 'evidence-inference' already exists and is not an empty directory.


In [1]:
from process_evidence_inference import read_docs

In [2]:
docs = read_docs(abst_only=True)

Reading prompts + articles


AttributeError: module 'preprocessor' has no attribute 'read_prompts'

In [ ]:
from bisect import *
def find_ge(a, x):
    'Find leftmost item greater than or equal to x'
    i = bisect_left(a, x)
    return i
    
def find_le(a, x):
    'Find rightmost value less than or equal to x'
    i = bisect_right(a, x)
    if i:
        return i-1
    raise ValueError

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
from tqdm import tqdm
annotations = []
for d in tqdm(docs) :
    dn = nlp(d.text)
    text = " ".join([t.text for t in dn if t.text.strip() != ''])
    tidx = [t.idx for t in dn if t.text.strip() != '']
    for i, f in enumerate(d.frames) :
        annotation_id = str(d.id) + '_' + str(i)
        annotations.append({
            'annotation_id' : annotation_id,
            'doc_id' : d.id, 
            'document' : text,
            'rationale' : [(find_le(tidx, f.ev.i), find_ge(tidx, f.ev.f))],
            'label_id' : f.encoder_label[f.label],
            'query' : ' || '.join([f.i, f.c, f.o])
        })

100%|██████████| 1772/1772 [02:15<00:00, 13.06it/s]


In [ ]:
import pandas as pd
import numpy as np
annotations = pd.DataFrame(annotations)
label_vocab = dict(enumerate(np.unique(annotations.label_id.values)))
label_vocab = {v:k for k,v in label_vocab.items()}
annotations["label"] = annotations.label_id.apply(lambda x: label_vocab[x])

In [ ]:
train_ids = [int(x) for x in open('evidence-inference/annotations/splits/train_article_ids.txt').read().strip().split()]
dev_ids = [int(x) for x in open('evidence-inference/annotations/splits/validation_article_ids.txt').read().strip().split()]
test_ids = [int(x) for x in open('evidence-inference/annotations/splits/test_article_ids.txt').read().strip().split()]

In [ ]:
train = annotations[annotations.doc_id.isin(train_ids)]
train["exp_split"] = "train"
dev = annotations[annotations.doc_id.isin(dev_ids)]
dev["exp_split"] = "dev"
test = annotations[annotations.doc_id.isin(test_ids)]
test["exp_split"] = "test"

import os
os.makedirs('data', exist_ok=True)
train.to_csv("data/train.csv", index = False)
dev.to_csv("data/dev.csv", index = False)
test.to_csv("data/test.csv", index = False)

/home/kokos/.conda/envs/requirements_file/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/kokos/.conda/envs/requirements_file/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/kokos/.conda/envs/requirements_file/lib/python3.5/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer